# Preprocessing and Counting PML Bodies

In [10]:
import json
import numpy as np
from scipy.spatial import distance
from skimage import io  # Importing image loading library
import os
import cv2
import pandas as pd

In [11]:
json_path = '/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/input/annotations/combined_annotations.json'

In [28]:
# Load JSON file
with open(json_path) as f:
    data = json.load(f)

# Initialize lists to store results for each image
inside_counts = []
outside_counts = []
mean_distances = []
variance_distances = []
inside_densities = []
outside_densities = []

In [29]:
df = pd.DataFrame(data)
df

,image_name,label,nucleus_mask,dots
0,flattened_position_40-1_C1,not arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 174, 'y': 504..."
1,flattened_position_37-1_C1,not arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 388, 'y': 511..."
2,flattened_position_36-1_C1,not arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 468, 'y': 427..."
3,flattened_position_10_C1,not arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 469, 'y': 511..."
4,flattened_position_6_C1,not arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 223, 'y': 511..."
...,...,...,...,...
155,flattened_position_15_C1,arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 394, 'y': 441..."
156,flattened_position_40_C1,arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 317, 'y': 465..."
157,flattened_position_23_C1,arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 312, 'y': 507..."
158,flattened_position_52_C1,arsenic,/Users/pallavisingh/Library/CloudStorage/OneDr...,"[{'dot_id': 1, 'position': {'x': 429, 'y': 423..."


In [21]:
import cv2
import numpy as np
from scipy.ndimage import distance_transform_edt

def analyze_pml_bodies(data):
    """
    Analyzes PML bodies in cell nuclei, calculating various metrics for both
    arsenic-treated and non-arsenic-treated samples.
    """
    # Initialize dictionaries to store results per nucleus
    nucleus_metrics = {
        'arsenic': [],
        'non_arsenic': []
    }
    
    # Initialize summary statistics
    summary_stats = {
        'arsenic': {
            'per_nucleus': {
                'mean_inside_count': 0,
                'mean_outside_count': 0,
                'mean_inside_density': 0,
                'mean_boundary_distance': 0
            },
            'total': {
                'nucleus_count': 0,
                'total_inside_dots': 0,
                'total_outside_dots': 0
            }
        },
        'non_arsenic': {
            'per_nucleus': {
                'mean_inside_count': 0,
                'mean_outside_count': 0,
                'mean_inside_density': 0,
                'mean_boundary_distance': 0
            },
            'total': {
                'nucleus_count': 0,
                'total_inside_dots': 0,
                'total_outside_dots': 0
            }
        }
    }
    
    # Process each image
    for entry in data:
        image_name = entry['image_name']
        is_arsenic = entry.get('label', '').strip().lower() == 'arsenic'
        treatment_type = 'arsenic' if is_arsenic else 'non_arsenic'
        
        # Read the nucleus mask
        nucleus_mask_path = entry.get('nucleus_mask')
        if nucleus_mask_path is None:
            print(f"Warning: No nucleus mask for {image_name}")
            continue
            
        nucleus_mask = cv2.imread(nucleus_mask_path, cv2.IMREAD_GRAYSCALE)
        if nucleus_mask is None:
            print(f"Warning: Could not read nucleus mask from {nucleus_mask_path}")
            continue
            
        nucleus_mask = (nucleus_mask > 0).astype(np.uint8)
        
        # Calculate nuclear area
        nuclear_area = np.sum(nucleus_mask)
        
        # Initialize metrics for this nucleus
        nucleus_data = {
            'image_name': image_name,
            'nuclear_area': nuclear_area,
            'inside_dots': [],
            'outside_dots': [],
            'boundary_distances': [],
            'inside_count': 0,
            'outside_count': 0
        }
        
        # Calculate distance transform for boundary analysis
        dist_transform = distance_transform_edt(nucleus_mask)
        
        # Process dots
        for dot in entry['dots']:
            x, y = int(dot['position']['x']), int(dot['position']['y'])
            is_inside = dot['location'].strip() == 'inside_nucleus'
            
            if is_inside:
                nucleus_data['inside_dots'].append((x, y))
                nucleus_data['inside_count'] += 1
                # Calculate distance to nuclear boundary
                boundary_dist = dist_transform[y, x]
                nucleus_data['boundary_distances'].append(boundary_dist)
            else:
                nucleus_data['outside_dots'].append((x, y))
                nucleus_data['outside_count'] += 1
        
        # Calculate density (PML bodies per unit area)
        nucleus_data['inside_density'] = (nucleus_data['inside_count'] / nuclear_area) if nuclear_area > 0 else 0
        
        # Calculate mean boundary distance for this nucleus
        nucleus_data['mean_boundary_distance'] = (
            np.mean(nucleus_data['boundary_distances']) if nucleus_data['boundary_distances'] else 0
        )
        
        # Add to collection
        nucleus_metrics[treatment_type].append(nucleus_data)
        
        # Update summary totals
        summary_stats[treatment_type]['total']['nucleus_count'] += 1
        summary_stats[treatment_type]['total']['total_inside_dots'] += nucleus_data['inside_count']
        summary_stats[treatment_type]['total']['total_outside_dots'] += nucleus_data['outside_count']
    
    # Calculate summary statistics for each treatment
    for treatment in ['arsenic', 'non_arsenic']:
        if nucleus_metrics[treatment]:
            nuclei = nucleus_metrics[treatment]
            stats = summary_stats[treatment]['per_nucleus']
            
            # Calculate means per nucleus
            stats['mean_inside_count'] = np.mean([n['inside_count'] for n in nuclei])
            stats['mean_outside_count'] = np.mean([n['outside_count'] for n in nuclei])
            stats['mean_inside_density'] = np.mean([n['inside_density'] for n in nuclei])
            stats['mean_boundary_distance'] = np.mean([n['mean_boundary_distance'] for n in nuclei])
            
            # Add standard deviations
            stats['std_inside_count'] = np.std([n['inside_count'] for n in nuclei])
            stats['std_outside_count'] = np.std([n['outside_count'] for n in nuclei])
            stats['std_inside_density'] = np.std([n['inside_density'] for n in nuclei])
            stats['std_boundary_distance'] = np.std([n['mean_boundary_distance'] for n in nuclei])
    
    # Create detailed results dictionary
    results = {
        'summary_statistics': summary_stats,
        'per_nucleus_data': nucleus_metrics
    }
    
    return results

def print_analysis_results(results):
    """
    Prints a formatted summary of the PML body analysis results.
    """
    print("\nAnalysis Summary:")
    
    # Print results for each treatment type
    for treatment in ['arsenic', 'non_arsenic']:
        print(f"\n{treatment.upper()} TREATMENT:")
        print("-" * 50)
        
        stats = results['summary_statistics'][treatment]
        per_nucleus = stats['per_nucleus']
        totals = stats['total']
        
        # Print total counts
        print(f"Total nuclei analyzed: {totals['nucleus_count']}")
        print(f"Total PML bodies inside nuclei: {totals['total_inside_dots']}")
        print(f"Total PML bodies outside nuclei: {totals['total_outside_dots']}")
        
        # Print per-nucleus statistics
        print("\nPer Nucleus Statistics:")
        print(f"Mean PML bodies inside: {per_nucleus['mean_inside_count']:.2f} ± {per_nucleus['std_inside_count']:.2f}")
        print(f"Mean PML bodies outside: {per_nucleus['mean_outside_count']:.2f} ± {per_nucleus['std_outside_count']:.2f}")
        print(f"Mean PML density inside nucleus: {per_nucleus['mean_inside_density']:.5f} ± {per_nucleus['std_inside_density']:.5f}")
        print(f"Mean distance to nuclear boundary: {per_nucleus['mean_boundary_distance']:.2f} ± {per_nucleus['std_boundary_distance']:.2f}")
        
        # Print individual nucleus data if desired
        print("\nIndividual Nucleus Data:")
        for i, nucleus in enumerate(results['per_nucleus_data'][treatment][:5], 1):  # Show first 5 nuclei
            print(f"\nNucleus {i} ({nucleus['image_name']}):")
            print(f"  Inside count: {nucleus['inside_count']}")
            print(f"  Outside count: {nucleus['outside_count']}")
            print(f"  Inside density: {nucleus['inside_density']:.5f}")
            print(f"  Mean boundary distance: {nucleus['mean_boundary_distance']:.2f}")
        
        if len(results['per_nucleus_data'][treatment]) > 5:
            print("\n... (showing first 5 nuclei only)")



In [22]:
#Example usage:
# data = [...] # Your input data
results = analyze_pml_bodies(data)
print_analysis_results(results)


Analysis Summary:

ARSENIC TREATMENT:
--------------------------------------------------
Total nuclei analyzed: 80
Total PML bodies inside nuclei: 4412
Total PML bodies outside nuclei: 782

Per Nucleus Statistics:
Mean PML bodies inside: 55.15 ± 21.41
Mean PML bodies outside: 9.78 ± 4.57
Mean PML density inside nucleus: 0.00295 ± 0.00079
Mean distance to nuclear boundary: 12.70 ± 1.24

Individual Nucleus Data:

Nucleus 1 (flattened_position_80_C1):
  Inside count: 71
  Outside count: 15
  Inside density: 0.00296
  Mean boundary distance: 13.26

Nucleus 2 (flattened_position_10_C1):
  Inside count: 45
  Outside count: 11
  Inside density: 0.00212
  Mean boundary distance: 11.42

Nucleus 3 (flattened_position_73_C1):
  Inside count: 35
  Outside count: 11
  Inside density: 0.00510
  Mean boundary distance: 12.06

Nucleus 4 (flattened_position_61_C1):
  Inside count: 38
  Outside count: 4
  Inside density: 0.00275
  Mean boundary distance: 13.32

Nucleus 5 (flattened_position_6_C1):
  In